In [1]:
import pymysql
import pandas as pd

In [2]:
data = pd.read_csv('sharks_messy.csv', engine='python')
display(data.head())

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


In [3]:
print('shape:', data.shape)
print('types:\n\n', data.dtypes)
print('Null values:\n\n', data.isna().sum())

shape: (5992, 24)
types:

 Case Number               object
Date                      object
Year                       int64
Type                      object
Country                   object
Area                      object
Location                  object
Activity                  object
Name                      object
Sex                       object
Age                       object
Injury                    object
Fatal (Y/N)               object
Time                      object
Species                   object
Investigator or Source    object
pdf                       object
href formula              object
href                      object
Case Number.1             object
Case Number.2             object
original order             int64
Unnamed: 22               object
Unnamed: 23               object
dtype: object
Null values:

 Case Number                  0
Date                         0
Year                         0
Type                         0
Country                     

In [4]:
#printing info to evaluate if dropping
filtered_df = data[data['Unnamed: 22'].notnull()]
filtered_df1 = data[data['Unnamed: 23'].notnull()]
display(filtered_df, filtered_df1)
display(data[(data['Case Number'] != data['Case Number.1']) | (data['Case Number'] != data['Case Number.2'])][['Case Number', 'Case Number.1', 'Case Number.2']])
display((data[data['href formula'] != data['href']][['href formula', 'href']]).head())
display((data[data['href formula'] != data['href']][['href formula', 'href']]).count())


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
1247,2006.06.18,18-Jun-06,2006,Unprovoked,BRAZIL,Pernambuco,"Punta Del Chifre Beach, Olinda",Body boarding,Humberto Pessoa Batista,M,...,NaN,globalsurfnews.com,2006.06.18-Batista.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2006.06.18,2006.06.18,4746,stopped here,NaN


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
4182,1952.07.13,13-Jul-52,1952,Provoked,USA,California,"San Diego, San Diego County",Fishing,"Gerald Howard, on board sportsfishing boat Ter...",M,...,NaN,"L.A. Times, 7/14/1952",1952.07.13-Howard.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1952.07.13,1952.07.13,1811,NaN,Teramo
5585,1879.03.10,10-Mar-1879,1879,Invalid,AUSTRALIA,New South Wales,Near Sydney,The steamship Bonnie Dundee lost in collision,Cabin boy of the Bonnie Dundee,M,...,NaN,"Star, 3/22/1879",1879.03.10-Bonnie-Dundee.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1879.03.10,1879.03.10,408,NaN,change filename


,Case Number,Case Number.1,Case Number.2
4,2016.09.15,2016.09.16,2016.09.15
33,2016.07.14.4,2016.07.14.R,2016.07.14.4
97,2016.01.24.b,2015.01.24.b,2016.01.24.b
116,2015.12.23,2015.11.07,2015.12.23
121,2015.10.28.a,2015.10.28,2015.10.28.a
169,2015.07-10,2015.07.10,2015.07.10
3296,1967.07.05,1967/07.05,1967.07.05
3569,"1962,08.30.b",1962.08.30.b,"1962,08.30.b"
3654,1961.09.02.R,"1961.09,06.R",1961.09.02.R
4177,1952.08.05,1952.08.04,1952.08.05


,href formula,href
20,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
27,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
61,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
107,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
114,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...


href formula    53
href            51
dtype: int64

In [5]:
#dropping columns with no relevant info. Hypothesis not yet formulated. Just understanding the data and eliminating generic info. 
"""
1. Unnamed: 23 = All nulls except 1 showing: Teramo change filename
2. Unnamed: 22 = All nulls except 1 showing: stopped here
3. pdf: includes the data we already have in the table (further explained). Since we cannot work with this data = drop. 
In case is needed we can reference back to the original dataset.
4. Case Number == Case Number.1 == Case Number.2. Hay 13/5992 casos en los que son diferentes. Es una muestra muy peque
5. href formula, href and pdf mostly include the same pdf. There are around 53/5992 which have different values but after cheking
some of them, they lead to the same website. Since the sample is very small and will not affect significatively the results,
I'll eliminate the columns.

"""

data_clean = data.drop(['Unnamed: 23','Unnamed: 22', 'pdf', 'Case Number','Case Number.1', 'Case Number.2', 'href', 'href formula'], axis=1)


display(data_clean.head(100))


,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,original order
0,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,13h00,NaN,"Orlando Sentinel, 9/19/2016",5993
1,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,NaN,"Orlando Sentinel, 9/19/2016",5992
2,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,10h43,NaN,"Orlando Sentinel, 9/19/2016",5991
3,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,"The Age, 9/18/2016",5990
4,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,"The Age, 9/16/2016",5989
5,15-Sep-16,2016,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,Occupant: Ben Stratton,NaN,NaN,Shark rammed boat. No injury to occupant,N,NaN,NaN,"West Australian, 9/15/2016",5988
6,11-Sep-16,2016,Unprovoked,USA,Florida,"Ponte Vedra, St. Johns County",Wading,male,M,60s,Minor injury to arm,N,15h15,3' to 4' shark,"News4Jax, 9/11/2016",5987
7,07-Sep-16,2016,Unprovoked,USA,Hawaii,"Makaha, Oahu",Swimming,female,F,51,Severe lacerations to shoulder & forearm,N,14h30,"Tiger shark, 10?","Hawaii News Now, 9/7/2016",5986
8,06-Sep-16,2016,Unprovoked,NEW CALEDONIA,North Province,Koumac,Kite surfing,David Jewell,M,50,FATAL,Y,15h40,NaN,"TVANouvelles, 9/6/2016",5985
9,05-Sep-16,2016,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,12,Lacerations & punctures to lower right leg,N,Late afternoon,NaN,"C. Creswell, GSAF",5984


In [6]:
#Hipotesis
'''

Hipotesis: Somos los humanos las nuevas focas de los tiburones? 
Nos estan cazando o les estamos obligando a desplazarse de su habitat?. 

Para ello, asumo que si el tiburon se desplaza en verano puede que su idea sea la de cazar 
ya que hay mayor cantidad de personas en las playas y las convertimos en comederos. 
Si el tiburon se mueve por la orilla en invierno, puede que les estemos desplazando de su habitat.

A) Si el numero de ataques es similar en invierno que en verano = les estamos desplazando. No hay motivo alguno para que 
esten en la orilla. Hay menor cantidad de comida y ellos tienen menos movildiad.
B) Si ataques en verano > ataques en invierno == somos focas


KEY VALUES: SURFEROS (TAMBIEN PRACTICAN SURF EN INVIERNO. LUGAR (FECHAS DE VERANO DISTINTAS), NAME (TURISTA DE VACACIONES)
INJURY (EL TIBURON ESTABA JUGANDO VS ATACA *LEER IDEAS, ))
'''

data_hypothesis = data_clean.drop(['Investigator or Source', 'original order'], axis=1)

In [7]:
display(data_hypothesis.head())

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species
0,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,13h00,NaN
1,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,NaN
2,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,10h43,NaN
3,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,NaN
4,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark


In [28]:
#Date-cleaning: limpiar fechas manteniendo solo los caracteres numericos
data_hypothesis.dtypes

data_hypothesis["Date"].replace(r'D+\s+','', inplace=True, regex=True)
data_hypothesis["Date"].replace(r'\d+-','', inplace=True, regex=True)
data_hypothesis["Date"].head()

#data_hypothesis["Date"].replace(r'\w\w\w-\d+','abcd123', inplace=True, regex=True)
#print(data_hypothesis["Date"].value_counts())

0    Sep-16
1    Sep-16
2    Sep-16
3    Sep-16
4    Sep-16
Name: Date, dtype: object

In [32]:
#puedo crear un diccionario para almacenar los abcd123:fecha. Elimino los que no sean abcd123 y luego le doy 
#el valor con el diccionario


"""
VOY POR AQUI: Me queda detectar las fechas Sep-16 y eliminar las todas las que no sean de esa forma.
"""
rows_to_clean= data_hypothesis[(data_hypothesis['Date']!=r'[\w\w\w-d+]')]

print(rows_to_clean)


SyntaxError: unexpected EOF while parsing (<ipython-input-32-3c20ae6616ba>, line 6)

In [33]:
"""
Quiero averiguar cuantos datos tengo en los que pueda obtener el mes y el anyo como minimo 
para asi saber si tengo suficiente info como para validar la hipotesis. Para ello voy a
utilizar un regex replace y los que cumplan el formato \w\w\w-\d+ les hare un replace por un valor abcd123 y luego 
hare un count abcd123
"""

#data_hypothesis["Date"].replace(r'\w\w\w-\d+','abcd123', inplace=True, regex=True)




'\nQuiero averiguar cuantos datos tengo en los que pueda obtener el mes y el anyo como minimo \npara asi saber si tengo suficiente info como para validar la hipotesis. Para ello voy a\nutilizar un regex replace y los que cumplan el formato \\w\\w\\w-\\d+ les hare un replace por un valor abcd123 y luego \nhare un count abcd123\n'

In [ ]:
#todaytime pandas para convertir sep-2018 a 09-2018

In [8]:
#ideas: 
"""

1. Separate nocturnal species (people barely have a bath at night)
2. A very small percentage of shark species, about two dozen, are considered potentially 
dangerous to humans because of their size and dentition.
3. When a shark grabs a person by the hand/wrist and tows them along the surface, tosses a surfboard 
(or a Frisbee as in case 1968.08.24) it is probably "play behavior", not aggression. 
How can case GSAF 1971.04.11 which the swimmer was repeatedly bitten by a large shark and case 1985.01.04 in which 
the diver's injury necessitated a Band-aid be compared? 
4. Global Shark Attack File believes the only way to sort fact from hype is by forensic examination of each incident. 
5. Sexo: Male = mas comida porque somos mas grandes. Lo mismo con la edad.
6. fatal: quedan restos de comida? defienden su territorio
7. El nombre no lo elimino ya que me puede dar el genero en caso de que falte. 
8. Species me sirve para saber si el ataque ha sido con intencion de alimenartse (carnivoro), o defensa (herbivoro)
"""







'\n\n1. Separate nocturnal species (people barely have a bath at night)\n2. A very small percentage of shark species, about two dozen, are considered potentially \ndangerous to humans because of their size and dentition.\n3. When a shark grabs a person by the hand/wrist and tows them along the surface, tosses a surfboard \n(or a Frisbee as in case 1968.08.24) it is probably "play behavior", not aggression. \nHow can case GSAF 1971.04.11 which the swimmer was repeatedly bitten by a large shark and case 1985.01.04 in which \nthe diver\'s injury necessitated a Band-aid be compared? \n4. Global Shark Attack File believes the only way to sort fact from hype is by forensic examination of each incident. \n5. Sexo: Male = mas comida porque somos mas grandes. Lo mismo con la edad.\n6. fatal: quedan restos de comida? defienden su territorio\n7. El nombre no lo elimino ya que me puede dar el genero en caso de que falte. \n8. Species me sirve para saber si el ataque ha sido con intencion de alimen